In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json
from PIL import Image
import keras
import os
import cv2

In [ ]:
root_dir = '/Users/Frost/Desktop/insulin_calculator_data/recognition_session_data/2020_1/8/19_28_31_4F40CE59-DD67-4A88-B572-A64497592EAE'
image_path =  os.path.join(root_dir, 'image.jpg')
# image_path = '/Volumes/tsanchen/carbs_estimate/error_tolerance_test/test_3/donut/roll/case_17/11_47_58_6B735A73-D166-4FB3-95BF-F17BBD4A1E93/image.jpg'
peripheral_path = os.path.join(root_dir, 'peripheral.json')

model_path = '/Users/Frost/Desktop/lovasz_inceptionresnetv2_unet.hdf5'

UNIFIED_IMAGE_SIZE = (512, 512)

def center_crop(array):
    assert len(array.shape) >= 2
    if array.shape[0] == array.shape[1]:
        return array
    shape_difference = abs(array.shape[0] - array.shape[1])
    offset = shape_difference // 2
    if array.shape[0] > array.shape[1]:
        return array[offset:array.shape[1] + offset, :]
    else:
        return array[:, offset:array.shape[0] + offset]

def center_normalize(image):
    mean = np.mean(cv2.resize(image, (512, 512)), axis=(0, 1))
    std = np.std(cv2.resize(image, (512, 512)), axis=(0, 1))
    return (image - mean) / std

In [ ]:
keras.losses.lovasz_hinge = keras.losses.binary_crossentropy
model = keras.models.load_model(model_path)

In [ ]:
with open(peripheral_path) as in_file:
    json_content = json.loads(in_file.read())
    depth_map = np.array(json_content['depth_data'])
image = np.array(Image.open(image_path))
plt.imshow(image)

In [ ]:
plt.imshow(depth_map)
plt.colorbar()

In [ ]:
mask = np.reshape(
    model.predict(
        np.reshape(
            center_normalize(
                cv2.resize(
                    center_crop(
                        image
                    ), 
                    UNIFIED_IMAGE_SIZE
                )
            ), 
            (1, *UNIFIED_IMAGE_SIZE, 3)
        )
    ), UNIFIED_IMAGE_SIZE
)
mask[mask < 0] = 0
mask[mask > 0] = 1
plt.imshow(cv2.resize(center_crop(image), (512, 512)))
plt.imshow(mask, alpha=0.5)
plt.colorbar()